In [2]:
import uproot
import uproot_methods
import pandas
import coffea.processor as processor
import numpy as np
from awkward import JaggedArray
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray

In [3]:
def getLorentz(x):
    objs = []
    for i in range(x['nGenPart']):
        if not ((abs(x['GenPart_pdgId'][i]) == 11 or abs(x['GenPart_pdgId'][i]) == 13) and x['GenPart_status'][i] == 1):
            continue
        objs.append(uproot_methods.TLorentzVector.from_ptetaphim(x['GenPart_pt'][i], 
                                        x['GenPart_eta'][i],
                                        x['GenPart_phi'][i],
                                        x['GenPart_mass'][i], 
            )
        )
    return objs

In [4]:
ewwz_file = uproot.open("Data/EWWZ_Nano/B0717A4C-F989-444A-BA62-304785265E1C.root")

In [5]:
readvars = ["*GenPart*", "MET*"]
tree = ewwz_file['Events']
df = processor.LazyDataFrame(tree, stride=200000, index=0)

In [6]:
pandas_df = tree.pandas.df(readvars, flatten=False)

In [7]:
good_events = (((np.abs(df['GenPart_pdgId']) == 11) | (np.abs(df['GenPart_pdgId']) == 13)) & (df['GenPart_status'] == 1) & (df['GenPart_pt'] > 10) & (np.abs(df['GenPart_eta']) < 2.5)).any()

In [8]:
slimmed_df = pandas_df[good_events]
slimmed_df

,nGenPart,GenPart_eta,GenPart_mass,GenPart_phi,GenPart_pt,GenPart_genPartIdxMother,GenPart_pdgId,GenPart_status,GenPart_statusFlags,MET_MetUnclustEnUpDeltaX,MET_MetUnclustEnUpDeltaY,MET_covXX,MET_covXY,MET_covYY,MET_phi,MET_pt,MET_significance,MET_sumEt,MET_fiducialGenPhi,MET_fiducialGenPt
entry,,,,,,,,,,,,,,,,,,,,
0,66,"[24064.0, -23296.0, -1.3007812, -3.4453125, 3....","[0.0, 0.0, 88.25, 80.5, 0.0, 0.0, 88.25, 80.5,...","[0.0, 0.0, 1.1523438, -1.828125, -1.703125, 2....","[0.0, 0.0, 102.0, 15.03125, 123.25, 48.375, 10...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 4, -1, ...","[1, 1, 23, -24, 2, 1, 23, -24, 23, -24, 23, -2...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[2433, 2433, 4481, 4481, 4481, 385, 257, 257, ...",2.761719,2.490234,766.0,-106.5000,746.0,1.502441,137.634064,26.421875,1574.0,1.467773,104.125000
1,35,"[23040.0, -23168.0, 1.2460938, -0.8925781, 1.1...","[0.0, 0.0, 90.0, 80.25, 0.0, 0.0, 90.0, 80.25,...","[0.0, 0.0, -0.69921875, 1.2109375, -1.3125, 3....","[0.0, 0.0, 39.125, 120.5, 103.75, 99.75, 36.62...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 10, 11,...","[4, -2, 23, 24, 3, -2, 23, 24, 23, 24, 23, 24,...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[10625, 2433, 4481, 4481, 4481, 385, 257, 257,...",-2.716797,-2.125000,351.0,-94.7500,644.0,-1.526611,22.172840,0.776855,1219.0,-1.958984,6.257812
4,54,"[24832.0, -23040.0, 2.515625, -1.2109375, -2.1...","[0.0, 0.0, 89.75, 91.25, 0.0, 0.0, 89.75, 91.2...","[0.0, 0.0, 1.4023438, 1.03125, -1.640625, -1.9...","[0.0, 0.0, 92.25, 84.0, 25.125, 149.5, 89.75, ...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 10, 11,...","[2, 2, 23, 24, 2, 1, 23, 24, 23, 24, 23, 24, 2...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[2433, 2433, 4481, 4481, 385, 4481, 257, 257, ...",-4.386719,-0.515137,880.0,138.5000,1368.0,1.280762,103.878494,7.691406,2626.0,1.722656,79.687500
5,55,"[24384.0, -22976.0, 2.6953125, -0.546875, 2.46...","[0.0, 0.0, 94.25, 82.0, 0.0, 0.0, 94.25, 82.0,...","[0.0, 0.0, -2.1015625, 0.953125, -1.5429688, 1...","[0.0, 0.0, 144.5, 99.0, 83.5, 127.75, 94.25, 1...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 10, 11,...","[1, -1, 23, -24, 2, -1, 23, -24, 23, -24, 23, ...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[10625, 2433, 4481, 4481, 4481, 385, 257, 257,...",3.675781,0.787109,634.0,-67.5000,1036.0,-2.416992,43.381191,2.671875,1942.0,1.827148,48.531250
6,34,"[23360.0, -23296.0, -0.6464844, 1.8867188, 2.0...","[0.0, 0.0, 91.5, 90.0, 0.0, 0.0, 91.5, 90.0, 9...","[0.0, 0.0, -3.109375, -2.96875, 2.5390625, -0....","[0.0, 0.0, 172.5, 64.5, 67.5, 293.0, 167.0, 53...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 10, 10,...","[2, 2, 23, 24, 2, 1, 23, 24, 23, 24, 23, 24, -...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 62, 6...","[2433, 2433, 4481, 4481, 385, 4481, 257, 257, ...",0.233154,0.472900,692.0,-187.0000,640.0,-2.229492,58.217319,7.097656,1824.0,-2.259766,42.718750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,55,"[22848.0, -23616.0, -0.1694336, -2.765625, -0....","[0.0, 0.0, 90.75, 80.5, 0.0, 0.0, 90.75, 80.5,...","[0.0, 0.0, -1.8789062, -1.875, 0.15478516, 1.8...","[0.0, 0.0, 148.5, 39.125, 102.25, 169.0, 145.5...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 4, 10, ...","[-3, 2, 23, 24, 1, -3, 23, 24, 23, 24, 23, 24,...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[2433, 10625, 4481, 4481, 4481, 385, 257, 257,...",1.935547,2.322266,770.0,-30.4375,1176.0,-1.035156,53.816952,2.716797,1848.0,-0.920898,47.468750
17996,44,"[24256.0, -22848.0, 2.078125, 2.7109375, 2.085...","[0.0, 0.0, 95.25, 80.25, 0.0, 0.0, 95.25, 80.2...","[0.0, 0.0, -0.48242188, -1.0, 2.71875, 2.58593...","[0.0, 0.0, 240.0, 23.9375, 53.875, 207.5, 235....","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 10, 11,...","[1, 2, 23, -24, 2, 2, 23, -24, 23, -24, 23, -2...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[10625, 2433, 4481, 4481, 385, 385, 257, 257, ...",6.398438,-3.535156,1636.0,-532.0000,1188.0,0.488708,95.190277,9.429

In [13]:
pandas.options.mode.chained_assignment = None
slimmed_df["leps"] = slimmed_df.apply(lambda x: getLorentz(x), axis=1)

In [29]:
slimmed_df[slimmed_df["leps"].apply(lambda x: len(x) >= 3)]

,nGenPart,GenPart_eta,GenPart_mass,GenPart_phi,GenPart_pt,GenPart_genPartIdxMother,GenPart_pdgId,GenPart_status,GenPart_statusFlags,MET_MetUnclustEnUpDeltaX,...,MET_covXY,MET_covYY,MET_phi,MET_pt,MET_significance,MET_sumEt,MET_fiducialGenPhi,MET_fiducialGenPt,leps,lep1
entry,,,,,,,,,,,,,,,,,,,,,
0,66,"[24064.0, -23296.0, -1.3007812, -3.4453125, 3....","[0.0, 0.0, 88.25, 80.5, 0.0, 0.0, 88.25, 80.5,...","[0.0, 0.0, 1.1523438, -1.828125, -1.703125, 2....","[0.0, 0.0, 102.0, 15.03125, 123.25, 48.375, 10...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 4, -1, ...","[1, 1, 23, -24, 2, 1, 23, -24, 23, -24, 23, -2...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[2433, 2433, 4481, 4481, 4481, 385, 257, 257, ...",2.761719,...,-106.5000,746.0,1.502441,137.634064,26.421875,1574.0,1.467773,104.12500,"[TLorentzVector(10.156, -1.043, 1.6562, 0), TL...","TLorentzVector(10.156, -1.043, 1.6562, 0)"
4,54,"[24832.0, -23040.0, 2.515625, -1.2109375, -2.1...","[0.0, 0.0, 89.75, 91.25, 0.0, 0.0, 89.75, 91.2...","[0.0, 0.0, 1.4023438, 1.03125, -1.640625, -1.9...","[0.0, 0.0, 92.25, 84.0, 25.125, 149.5, 89.75, ...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 10, 11,...","[2, 2, 23, 24, 2, 1, 23, 24, 23, 24, 23, 24, 2...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[2433, 2433, 4481, 4481, 385, 4481, 257, 257, ...",-4.386719,...,138.5000,1368.0,1.280762,103.878494,7.691406,2626.0,1.722656,79.68750,"[TLorentzVector(45.125, -1.6328, -0.40234, 0),...","TLorentzVector(45.125, -1.6328, -0.40234, 0)"
5,55,"[24384.0, -22976.0, 2.6953125, -0.546875, 2.46...","[0.0, 0.0, 94.25, 82.0, 0.0, 0.0, 94.25, 82.0,...","[0.0, 0.0, -2.1015625, 0.953125, -1.5429688, 1...","[0.0, 0.0, 144.5, 99.0, 83.5, 127.75, 94.25, 1...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 10, 11,...","[1, -1, 23, -24, 2, -1, 23, -24, 23, -24, 23, ...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[10625, 2433, 4481, 4481, 4481, 385, 257, 257,...",3.675781,...,-67.5000,1036.0,-2.416992,43.381191,2.671875,1942.0,1.827148,48.53125,"[TLorentzVector(60.125, 3.0156, -1.3516, 0), T...","TLorentzVector(60.125, 3.0156, -1.3516, 0)"
6,34,"[23360.0, -23296.0, -0.6464844, 1.8867188, 2.0...","[0.0, 0.0, 91.5, 90.0, 0.0, 0.0, 91.5, 90.0, 9...","[0.0, 0.0, -3.109375, -2.96875, 2.5390625, -0....","[0.0, 0.0, 172.5, 64.5, 67.5, 293.0, 167.0, 53...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 10, 10,...","[2, 2, 23, 24, 2, 1, 23, 24, 23, 24, 23, 24, -...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 62, 6...","[2433, 2433, 4481, 4481, 385, 4481, 257, 257, ...",0.233154,...,-187.0000,640.0,-2.229492,58.217319,7.097656,1824.0,-2.259766,42.71875,"[TLorentzVector(26.562, -0.37012, -2.8438, 0),...","TLorentzVector(26.562, -0.37012, -2.8438, 0)"
7,52,"[24256.0, -24192.0, -1.9804688, -1.2890625, -2...","[0.0, 0.0, 82.5, 80.25, 0.0, 0.0, 82.5, 80.25,...","[0.0, 0.0, 2.8046875, -0.63671875, 0.44921875,...","[0.0, 0.0, 104.25, 106.0, 98.25, 76.5, 104.25,...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 10, 11,...","[1, 2, 23, 24, 1, 1, 23, 24, 23, 24, 23, 24, 2...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[2433, 10625, 4481, 4481, 385, 385, 257, 257, ...",-7.042969,...,-38.3750,906.0,0.127502,30.481129,0.851074,2368.0,0.295898,51.06250,"[TLorentzVector(20.688, -2.25, -2.7188, 0), TL...","TLorentzVector(20.688, -2.25, -2.7188, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17994,44,"[22976.0, -24064.0, -3.4375, -1.0585938, -3.28...","[0.0, 0.0, 90.5, 79.5, 0.0, 0.0, 90.5, 79.5, 9...","[0.0, 0.0, -2.8046875, -1.1328125, 1.0546875, ...","[0.0, 0.0, 5.796875, 140.0, 71.75, 111.0, 5.65...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 10, 11,...","[-1, 2, 23, 24, 2, -2, 23, 24, 23, 24, 23, 24,...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[10625, 2433, 4481, 4481, 385, 4481, 257, 257,...",2.273438,...,16.7500,730.0,-1.050781,149.089554,30.406250,1706.0,-1.207031,132.12500,"[TLorentzVector(20.25, -0.00042343, -0.74219, ...","TLorentzVector(20.25, -0.00042343, -0.7421

In [14]:
for i in range(3):
    slimmed_df["lep%i" % (i+1)] = slimmed_df.apply(lambda x: x['leps'][i],
                                axis=1)

IndexError: ('list index out of range', 'occurred at index 1')

In [164]:
slimmed_df

,nGenPart,GenPart_eta,GenPart_mass,GenPart_phi,GenPart_pt,GenPart_genPartIdxMother,GenPart_pdgId,GenPart_status,GenPart_statusFlags,MET_MetUnclustEnUpDeltaX,...,MET_phi,MET_pt,MET_significance,MET_sumEt,MET_fiducialGenPhi,MET_fiducialGenPt,leps,lep1,lep2,lep3
entry,,,,,,,,,,,,,,,,,,,,,
0,66,"[24064.0, -23296.0, -1.3007812, -3.4453125, 3....","[0.0, 0.0, 88.25, 80.5, 0.0, 0.0, 88.25, 80.5,...","[0.0, 0.0, 1.1523438, -1.828125, -1.703125, 2....","[0.0, 0.0, 102.0, 15.03125, 123.25, 48.375, 10...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 4, -1, ...","[1, 1, 23, -24, 2, 1, 23, -24, 23, -24, 23, -2...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[2433, 2433, 4481, 4481, 4481, 385, 257, 257, ...",2.761719,...,1.502441,137.634064,26.421875,1574.0,1.467773,104.125000,"[TLorentzVector(10.156, -1.043, 1.6562, 0), TL...","TLorentzVector(10.156, -1.043, 1.6562, 0)","TLorentzVector(10.156, -1.043, 1.6562, 0)","TLorentzVector(10.156, -1.043, 1.6562, 0)"
1,35,"[23040.0, -23168.0, 1.2460938, -0.8925781, 1.1...","[0.0, 0.0, 90.0, 80.25, 0.0, 0.0, 90.0, 80.25,...","[0.0, 0.0, -0.69921875, 1.2109375, -1.3125, 3....","[0.0, 0.0, 39.125, 120.5, 103.75, 99.75, 36.62...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 10, 11,...","[4, -2, 23, 24, 3, -2, 23, 24, 23, 24, 23, 24,...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[10625, 2433, 4481, 4481, 4481, 385, 257, 257,...",-2.716797,...,-1.526611,22.172840,0.776855,1219.0,-1.958984,6.257812,"[TLorentzVector(132.5, -0.73828, 1.2305, 0)]","TLorentzVector(132.5, -0.73828, 1.2305, 0)","TLorentzVector(132.5, -0.73828, 1.2305, 0)","TLorentzVector(132.5, -0.73828, 1.2305, 0)"
4,54,"[24832.0, -23040.0, 2.515625, -1.2109375, -2.1...","[0.0, 0.0, 89.75, 91.25, 0.0, 0.0, 89.75, 91.2...","[0.0, 0.0, 1.4023438, 1.03125, -1.640625, -1.9...","[0.0, 0.0, 92.25, 84.0, 25.125, 149.5, 89.75, ...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 10, 11,...","[2, 2, 23, 24, 2, 1, 23, 24, 23, 24, 23, 24, 2...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[2433, 2433, 4481, 4481, 385, 4481, 257, 257, ...",-4.386719,...,1.280762,103.878494,7.691406,2626.0,1.722656,79.687500,"[TLorentzVector(45.125, -1.6328, -0.40234, 0),...","TLorentzVector(45.125, -1.6328, -0.40234, 0)","TLorentzVector(45.125, -1.6328, -0.40234, 0)","TLorentzVector(45.125, -1.6328, -0.40234, 0)"
5,55,"[24384.0, -22976.0, 2.6953125, -0.546875, 2.46...","[0.0, 0.0, 94.25, 82.0, 0.0, 0.0, 94.25, 82.0,...","[0.0, 0.0, -2.1015625, 0.953125, -1.5429688, 1...","[0.0, 0.0, 144.5, 99.0, 83.5, 127.75, 94.25, 1...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 10, 11,...","[1, -1, 23, -24, 2, -1, 23, -24, 23, -24, 23, ...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 44, 4...","[10625, 2433, 4481, 4481, 4481, 385, 257, 257,...",3.675781,...,-2.416992,43.381191,2.671875,1942.0,1.827148,48.531250,"[TLorentzVector(60.125, 3.0156, -1.3516, 0), T...","TLorentzVector(60.125, 3.0156, -1.3516, 0)","TLorentzVector(60.125, 3.0156, -1.3516, 0)","TLorentzVector(60.125, 3.0156, -1.3516, 0)"
6,34,"[23360.0, -23296.0, -0.6464844, 1.8867188, 2.0...","[0.0, 0.0, 91.5, 90.0, 0.0, 0.0, 91.5, 90.0, 9...","[0.0, 0.0, -3.109375, -2.96875, 2.5390625, -0....","[0.0, 0.0, 172.5, 64.5, 67.5, 293.0, 167.0, 53...","[-1, -1, 0, 0, 0, 0, 2, 3, 6, 7, 8, 9, 10, 10,...","[2, 2, 23, 24, 2, 1, 23, 24, 23, 24, 23, 24, -...","[21, 21, 22, 22, 23, 23, 44, 44, 44, 44, 62, 6...","[2433, 2433, 4481, 4481, 385, 4481, 257, 257, ...",0.233154,...,-2.229492,58.217319,7.097656,1824.0,-2.259766,42.718750,"[TLorentzVector(26.562, -0.37012, -2.8438, 0),...","TLorentzVector(26.562, -0.37012, -2.8438, 0)","TLorentzVector(26.562, -0.37012, -2.8438, 0)","TLorentzVector(26.562, -0.37012, -2.8438, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,55,"[22848.0, -23616.0, -0.1694336, -2.765625, -0....","[0.0, 0.0, 90.75, 80.5, 0.0, 0.0, 90.75, 80.5,...","[0.0, 0.0, -1.8789062, -1.875, 0.15478516, 1.8...","[0.0, 0.0, 148.5, 39.125, 102.25, 169.0, 145.5...","[-1, -1, 0, 0, 0, 

In [41]:
slimmed_df.apply(lambda x: x['nGenPart'] > 0 and x['GenPart_statusFlags'][0]<<1, axis=1)

entry
0         4866
1        21250
4         4866
5        21250
6         4866
         ...  
17995     4866
17996    21250
17997     4866
17998     4866
17999    21250
Length: 16294, dtype: int64